# Demonstrate How to Combine Multiple Tanks into a Network

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

import casadi as cas
from cas_models.discrete_time.models import StateSpaceModelDT
from cas_models.transformations import connect_systems
from cas_models.discrete_time.simulate import make_n_step_simulation_function_from_model
from feed_conc_ctrl import MixingTankModelDT, FlowMixerDT

In [2]:
plot_dir = Path("plots")
plot_dir.mkdir(exist_ok=True)

In [3]:
dt = 1
D = 5  # tank diameter [m]
n_tanks = 4
tank_names = [f"tank_{i+1}" for i in range(n_tanks)]

# Initialize tank system models
systems = [MixingTankModelDT(dt, D=D, name=name) for name in tank_names]
systems.append(FlowMixerDT(dt, n_in=2, name="mixer"))

def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

for sys in systems:
    print_sys_dimensions(sys)
    

tank_1 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']
tank_2 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']
tank_3 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']
tank_4 (3x3)
    input_names: ['v_dot_in', 'conc_in', 'v_dot_out']
    state_names: ['L', 'm']
   output_names: ['L', 'm', 'conc_out']
mixer (2x4)
    input_names: ['v_dot_in_1', 'conc_in_1', 'v_dot_in_2', 'conc_in_2']
    state_names: []
   output_names: ['v_dot_out', 'conc_out']


In [4]:
connections = {
    'tank_1_v_dot_out': ['tank_2_v_dot_in', 'tank_3_v_dot_in'],
    'tank_2_conc_in': 'tank_1_conc_out',
    'tank_3_conc_in': 'tank_1_conc_out',
    'mixer_conc_in_1': 'tank_2_conc_out',
    'mixer_conc_in_2': 'tank_3_conc_out',
    'mixer_v_dot_in_1': 'tank_2_v_dot_out',
    'mixer_v_dot_in_2': 'tank_3_v_dot_out',
    'tank_4_conc_in': 'mixer_conc_out',
    'tank_4_v_dot_in': 'mixer_v_dot_out',
}

model_class = StateSpaceModelDT
feed_tanks_system = connect_systems(
    systems, connections, model_class, name="tank_system_121", verbose_names=True
) 
print_sys_dimensions(feed_tanks_system)

tank_system_121 (14x7)
    input_names: ['tank_1_v_dot_in', 'tank_1_conc_in', 'tank_2_v_dot_in', 'tank_2_v_dot_out', 'tank_3_v_dot_in', 'tank_3_v_dot_out', 'tank_4_v_dot_out']
    state_names: ['tank_1_L', 'tank_1_m', 'tank_2_L', 'tank_2_m', 'tank_3_L', 'tank_3_m', 'tank_4_L', 'tank_4_m']
   output_names: ['tank_1_L', 'tank_1_m', 'tank_1_conc_out', 'tank_2_L', 'tank_2_m', 'tank_2_conc_out', 'tank_3_L', 'tank_3_m', 'tank_3_conc_out', 'tank_4_L', 'tank_4_m', 'tank_4_conc_out', 'mixer_v_dot_out', 'mixer_conc_out']


## Debug Function Formulas

In [5]:
# Debug: Analyze connection logic with symbolic inputs
print("=" * 80)
print("SYMBOLIC ANALYSIS OF CONNECTED SYSTEM")
print("=" * 80)

# Create symbolic inputs
t_sym = cas.SX.sym("t")
xk_sym = cas.SX.sym("xk", feed_tanks_system.n)
uk_sym = cas.SX.sym("uk", feed_tanks_system.nu)

# Get symbolic expressions for F and H
F_expr = feed_tanks_system.F(t_sym, xk_sym, uk_sym)
H_expr = feed_tanks_system.H(t_sym, xk_sym, uk_sym)

print(f"\nInput names ({feed_tanks_system.nu}): {feed_tanks_system.input_names}")
print(f"State names ({feed_tanks_system.n}): {feed_tanks_system.state_names}")  
print(f"Output names ({feed_tanks_system.ny}): {feed_tanks_system.output_names}")

print("\n" + "=" * 80)
print("STATE TRANSITION EQUATIONS (F)")
print("=" * 80)
for i, state_name in enumerate(feed_tanks_system.state_names):
    print(f"\n{state_name}(k+1) =")
    print(f"  {F_expr[i]}")

print("\n" + "=" * 80)
print("OUTPUT EQUATIONS (H) - Focus on key flows")
print("=" * 80)

# Show specific outputs we care about
key_outputs = {
    'mixer_v_dot_out': -2,  # Second to last output
    'mixer_conc_out': -1,   # Last output
    'tank_4_L': 9,          # tank_4_L
    'tank_4_m': 10,         # tank_4_m
}

for name, idx in key_outputs.items():
    print(f"\n{name} =")
    print(f"  {H_expr[idx]}")

feed_tanks_system.H

SYMBOLIC ANALYSIS OF CONNECTED SYSTEM

Input names (7): ['tank_1_v_dot_in', 'tank_1_conc_in', 'tank_2_v_dot_in', 'tank_2_v_dot_out', 'tank_3_v_dot_in', 'tank_3_v_dot_out', 'tank_4_v_dot_out']
State names (8): ['tank_1_L', 'tank_1_m', 'tank_2_L', 'tank_2_m', 'tank_3_L', 'tank_3_m', 'tank_4_L', 'tank_4_m']
Output names (14): ['tank_1_L', 'tank_1_m', 'tank_1_conc_out', 'tank_2_L', 'tank_2_m', 'tank_2_conc_out', 'tank_3_L', 'tank_3_m', 'tank_3_conc_out', 'tank_4_L', 'tank_4_m', 'tank_4_conc_out', 'mixer_v_dot_out', 'mixer_conc_out']

STATE TRANSITION EQUATIONS (F)

tank_1_L(k+1) =
  @1=(uk_2+uk_4), @2=19.635, @3=2, (xk_0+(0.166667*(((((uk_0-@1)/@2)+(@3*((uk_0-@1)/@2)))+(@3*((uk_0-@1)/@2)))+((uk_0-@1)/@2))))

tank_1_m(k+1) =
  @1=(uk_2+uk_4), @2=19.635, @3=((uk_0*uk_1)-((@1*xk_1)/(@2*xk_0))), @4=2, @5=0.5, @6=((uk_0*uk_1)-((@1*(xk_1+(@5*@3)))/(@2*(xk_0+(@5*((uk_0-@1)/@2)))))), @7=((uk_0*uk_1)-((@1*(xk_1+(@5*@6)))/(@2*(xk_0+(@5*((uk_0-@1)/@2)))))), (xk_1+(0.166667*(((@3+(@4*@6))+(@4*@7))+((u

Function(H:(t,xk[8],uk[7])->(yk[14]) SXFunction)

In [6]:
{name: 0.0 for name in feed_tanks_system.input_names}

{'tank_1_v_dot_in': 0.0,
 'tank_1_conc_in': 0.0,
 'tank_2_v_dot_in': 0.0,
 'tank_2_v_dot_out': 0.0,
 'tank_3_v_dot_in': 0.0,
 'tank_3_v_dot_out': 0.0,
 'tank_4_v_dot_out': 0.0}

In [7]:
# Test F and H functions
# F(t,xk[8],uk[7]) -> (xkp1[8])
# H(t,xk[8],uk[7]) -> (yk[14])

# Test 1 - no flows
t = 0.0  # irrelevant for this system
xk = pd.Series({
    'tank_1_L': 1.0,
    'tank_1_m': 0.5,
    'tank_2_L': 2.0,
    'tank_2_m': 0.6,
    'tank_3_L': 3.0,
    'tank_3_m': 0.7,
    'tank_4_L': 4.0,
    'tank_4_m': 0.8
}, name='xk')
uk = pd.Series({
    'tank_1_v_dot_in': 0.0,
    'tank_1_conc_in': 0.0,
    'tank_2_v_dot_in': 0.0,
    'tank_2_v_dot_out': 0.0,
    'tank_3_v_dot_in': 0.0,
    'tank_3_v_dot_out': 0.0,
    'tank_4_v_dot_out': 0.0
 }, name='uk')

xkp1 = feed_tanks_system.F(t, xk.to_list(), uk.to_list())
xkp1 = pd.Series(
    np.array(xkp1).squeeze(), 
    index=feed_tanks_system.state_names,
    name='xkp1'
)
print(pd.concat([xk, xkp1], axis=1))

# No change
assert (
    xkp1.to_dict() == xk.to_dict() 
)


           xk  xkp1
tank_1_L  1.0   1.0
tank_1_m  0.5   0.5
tank_2_L  2.0   2.0
tank_2_m  0.6   0.6
tank_3_L  3.0   3.0
tank_3_m  0.7   0.7
tank_4_L  4.0   4.0
tank_4_m  0.8   0.8


In [8]:
# Test 2 - flows with constant concentration
t = 0.0  # irrelevant for this system
density = 0.75
volume_per_m = np.pi * D**2/ 4

# Same concentration in all tanks
xk = pd.Series({
    'tank_1_L': 1.0,
    'tank_1_m': density * volume_per_m * 1.0,
    'tank_2_L': 2.0,
    'tank_2_m': density * volume_per_m * 2.0,
    'tank_3_L': 3.0,
    'tank_3_m': density * volume_per_m * 3.0,
    'tank_4_L': 4.0,
    'tank_4_m': density * volume_per_m * 4.0
}, name='xk')

# Equal flow in and out
uk = pd.Series({
    'tank_1_v_dot_in': 2.0,
    'tank_1_conc_in': density,
    'tank_2_v_dot_in': 1.0,
    'tank_2_v_dot_out': 1.0,
    'tank_3_v_dot_in': 1.0,
    'tank_3_v_dot_out': 1.0,
    'tank_4_v_dot_out': 2.0
 }, name='uk')

xkp1 = feed_tanks_system.F(t, xk.to_list(), uk.to_list())
xkp1 = pd.Series(
    np.array(xkp1).squeeze(), 
    index=feed_tanks_system.state_names,
    name='xkp1'
)
print(f"States:\n{pd.concat([xk, xkp1], axis=1)}")

# No changes - should be at steady state
assert xkp1.to_dict() == xk.to_dict(), "Test 2 FAILED: States changed in steady state!"
print("\n✓ Test 2 PASSED: Steady state maintained with equal flows in/out")

States:
                 xk       xkp1
tank_1_L   1.000000   1.000000
tank_1_m  14.726216  14.726216
tank_2_L   2.000000   2.000000
tank_2_m  29.452431  29.452431
tank_3_L   3.000000   3.000000
tank_3_m  44.178647  44.178647
tank_4_L   4.000000   4.000000
tank_4_m  58.904862  58.904862

✓ Test 2 PASSED: Steady state maintained with equal flows in/out


In [9]:
yk = feed_tanks_system.H(t, xk.to_list(), uk.to_list())
yk = pd.Series(
    np.array(yk).squeeze(), 
    index=feed_tanks_system.output_names,
    name='yk'
)
print(f"\nOutputs:\n{yk}")


Outputs:
tank_1_L            1.000000
tank_1_m           14.726216
tank_1_conc_out     0.750000
tank_2_L            2.000000
tank_2_m           29.452431
tank_2_conc_out     0.750000
tank_3_L            3.000000
tank_3_m           44.178647
tank_3_conc_out     0.750000
tank_4_L            4.000000
tank_4_m           58.904862
tank_4_conc_out     0.750000
mixer_v_dot_out     2.000000
mixer_conc_out      0.750000
Name: yk, dtype: float64


In [10]:
nT = 100
simulate = make_n_step_simulation_function_from_model(feed_tanks_system, nT)
simulate

Function(F_sim_100_steps:(t_eval[101],U[100x7],x0[8])->(X[101x8],Y[101x14]) SXFunction)

## Demo simulation

In [11]:
# Sample period
Ts = 1  # hours

# Evaluation times
t = Ts * np.arange(nT+1)

# Input signals
t_in = t[:-1]
U = np.zeros((nT, feed_tanks_system.nu))
U[t_in > 5, 0] = 10.0  # tank_1_v_dot_in
U[:, 1] = 0.5  # tank_1_conc_in
U[t_in > 10, 2] = 5.0  # tank_2_v_dot_in
U[t_in > 30, 3] = 5.0  # tank_2_v_dot_out
U[t_in > 20, 4] = 5.0  # tank_3_v_dot_in
U[t_in > 40, 5] = 5.0  # tank_3_v_dot_out
U[t_in > 50, 6] = 10.0  # tank_4_v_dot_out

# Initial conditions
x0 = [
    0.5,  # tank_1_L
    0,  # tank_1_m
    0.5,  # tank_2_L
    0,  # tank_2_m
    0.5,  # tank_3_L
    0,  # tank_3_m
    0.5,  # tank_4_L
    0,  # tank_4_m
]

X, Y = simulate(t, U, x0)

assert X.shape == (nT+1, feed_tanks_system.n)
assert Y.shape == (nT+1, feed_tanks_system.ny)

sim_results = {
    "inputs": pd.DataFrame(U, columns=feed_tanks_system.input_names),
    "states": pd.DataFrame(X, columns=feed_tanks_system.state_names),
    "outputs": pd.DataFrame(Y, columns=feed_tanks_system.output_names)
}
sim_results = pd.concat(sim_results, axis=1)
sim_results.head()


inputs                                                  \
  tank_1_v_dot_in tank_1_conc_in tank_2_v_dot_in tank_2_v_dot_out   
0             0.0            0.5             0.0              0.0   
1             0.0            0.5             0.0              0.0   
2             0.0            0.5             0.0              0.0   
3             0.0            0.5             0.0              0.0   
4             0.0            0.5             0.0              0.0   

                                                      states           \
  tank_3_v_dot_in tank_3_v_dot_out tank_4_v_dot_out tank_1_L tank_1_m   
0             0.0              0.0              0.0      0.5      0.0   
1             0.0              0.0              0.0      0.5      0.0   
2             0.0              0.0              0.0      0.5      0.0   
3             0.0              0.0              0.0      0.5      0.0   
4             0.0              0.0              0.0      0.5      0.0   

            ...  outputs                                                    \
  tank_2_L  ... tank_2_m tank_2_conc_out tank_3_L tank_3_m tank_3_conc_out   
0      0.5  ...      0.0             0.0      0.5      0.0             0.0   
1      0.5  ...      0.0             0.0      0.5      0.0             0.0   
2      0.5  ...      0.0             0.0      0.5      0.0             0.0   
3      0.5  ...      0.0             0.0      0.5      0.0             0.0   
4      0.5  ...      0.0             0.0      0.5      0.0             0.0   

                                                                    
  tank_4_L tank_4_m tank_4_conc_out mixer_v_dot_out mixer_conc_out  
0      0.5      0.0             0.0             0.0            NaN  
1      0.5      0.0             0.0             0.0            NaN  
2      0.5      0.0             0.0             0.0            NaN  
3      0.5      0.0             0.0             0.0            NaN  
4      0.5      0.0             0.0             0.0            NaN  

[5 rows x 29 columns]

In [12]:
flow_units = r'$m^3/h$'
conc_units = r'$tons/m^3$'
level_units = r'$m$'

units = {
    'tank_1_v_dot_in': flow_units, 
    'tank_1_v_dot_out': flow_units, 
    'tank_2_v_dot_in': flow_units, 
    'tank_2_v_dot_out': flow_units, 
    'tank_3_v_dot_in': flow_units,  
    'tank_3_v_dot_out': flow_units, 
    'tank_4_v_dot_in': flow_units, 
    'tank_4_v_dot_out': flow_units,
    'tank_1_conc_in': conc_units,
    'tank_1_conc_out': conc_units,
    'tank_2_conc_out': conc_units,
    'tank_3_conc_out': conc_units,
    'tank_4_conc_out': conc_units,
    'tank_1_L': level_units,
    'tank_2_L': level_units,
    'tank_3_L': level_units,
    'tank_4_L': level_units,
}

plot_info = {
    "Tank 1 Flows": {
        "Flow In": 'tank_1_v_dot_in',
    },
    "Tank 2 Flows": {
        "Flow In": 'tank_2_v_dot_in',
        "Flow Out": 'tank_2_v_dot_out'
    },
    "Tank 3 Flows": {
        "Flow In": 'tank_3_v_dot_in',
        "Flow Out": 'tank_3_v_dot_out'
    },
    "Tank 4 Flows": {
        "Flow Out": 'tank_4_v_dot_out'
    },
    "Tank Levels": {
        "Tank 1": 'tank_1_L',
        "Tank 2": 'tank_2_L',
        "Tank 3": 'tank_3_L',
        "Tank 4": 'tank_4_L'
    },
    "Tank Concentrations": {
        "Feed": 'tank_1_conc_in',
        "Tank 1": 'tank_1_conc_out',
        "Tank 2": 'tank_2_conc_out',
        "Tank 3": 'tank_3_conc_out',
        "Tank 4": 'tank_4_conc_out'
    },
}

In [13]:
def make_tsplots(
    data, plot_info, units=None, time_units='hours', time_label='Time ({time_units})'
):
    n_subplots = len(plot_info)
    width, height = 8, 1 + 1.5*n_subplots

    fig, axes = plt.subplots(n_subplots, 1, sharex=True, figsize=(width, height))
    for ax, (title, sub_plot_info) in zip(axes, plot_info.items()):
        for name, var_name in sub_plot_info.items():
            data[var_name].plot(ax=ax, label=name)
        ax.set_ylabel(units[var_name])
        ax.grid(True)
        ax.legend()
        ax.set_title(title)

    axes[-1].set_xlabel(time_label.format(time_units=time_units))

    return fig, axes

var_data = sim_results.droplevel(0, axis=1)
var_data = var_data.loc[:, ~var_data.columns.duplicated()]
fig, axes = make_tsplots(var_data, plot_info, units=units)
plt.tight_layout()
plt.savefig(plot_dir / "tank_network_simulation.png")
plt.close()